In [1]:
import os
WORKSPACE_DIR = "./workspace" 
print(f'WORKSPACE_DIR = {WORKSPACE_DIR}')
data_dir = os.path.join(WORKSPACE_DIR, "data")
os.makedirs(data_dir, exist_ok=True)

WORKSPACE_DIR = ./workspace


In [2]:
# %%capture
%pip install opendartreader

Note: you may need to restart the kernel to use updated packages.


In [3]:
DART_API_KEY = ""

### Download and Extract Company Disclosure from DART

In [4]:
import OpenDartReader

dart = OpenDartReader(DART_API_KEY) 

In [5]:
# 2016-10-01 ~ 2022-06-01 cgv 회사의 모든 사업보고서

cgv=dart.list('079160', start='2016-10-01', end='2022-06-01', kind='A')
cgv = cgv[cgv['report_nm'].str.contains('사업보고서')]
cgv

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
1,00303873,CJ CGV,079160,Y,사업보고서 (2021.12),20220321001182,CJ CGV,20220321,연
5,00303873,CJ CGV,079160,Y,사업보고서 (2020.12),20210319001272,CJ CGV,20210319,연
9,00303873,CJ CGV,079160,Y,사업보고서 (2019.12),20200330003232,CJ CGV,20200330,연
12,00303873,CJ CGV,079160,Y,[기재정정]사업보고서 (2018.12),20190702000269,CJ CGV,20190702,연
17,00303873,CJ CGV,079160,Y,사업보고서 (2017.12),20180330003675,CJ CGV,20180330,연
21,00303873,CJ CGV,079160,Y,사업보고서 (2016.12),20170331004598,CJ CGV,20170331,연


In [7]:
import time
import random
import requests
from bs4 import BeautifulSoup

data_cgv=''
data_dict_cgv = {}

for key,doc in cgv['rcept_no'].iteritems():
    sub_docs = dart.sub_docs(str(doc), match='이사의 경영진단 및 분석의견') # 검색
    url = sub_docs.url[sub_docs.index[0]] # 가장 잘 맞는거 뽑고
    page = requests.get(url) # url뽑고
    time.sleep(random.uniform(0.3,1.8))
    soup = BeautifulSoup(page.content, "html.parser") #페이지 불러오기
    raw_data = soup.find_all('p')
    for item in raw_data:
        data_cgv += item.getText()
        data_cgv = data_cgv.replace(u'\xa0',u'')
        data_dict_cgv[doc] = [data_cgv] # 딕셔너리 저장


In [8]:
import pandas as pd
cgvs = pd.DataFrame(data_dict_cgv)
cgvs = cgvs.transpose()
cgvs = cgvs.reset_index()
cgvs.rename(columns= {0:'text', 'index':'rcept_no'}, inplace=True)
cgvs

,rcept_no,text
0,20220321001182,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
1,20210319001272,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
2,20200330003232,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
3,20190702000269,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
4,20180330003675,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
5,20170331004598,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...


In [9]:
cgvs = pd.merge(cgv,cgvs)
cgvs.to_excel('cgvs.xlsx')
cgvs.to_csv("cgv.csv", mode='w')
cgvs

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm,text
0,00303873,CJ CGV,079160,Y,사업보고서 (2021.12),20220321001182,CJ CGV,20220321,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
1,00303873,CJ CGV,079160,Y,사업보고서 (2020.12),20210319001272,CJ CGV,20210319,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
2,00303873,CJ CGV,079160,Y,사업보고서 (2019.12),20200330003232,CJ CGV,20200330,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
3,00303873,CJ CGV,079160,Y,[기재정정]사업보고서 (2018.12),20190702000269,CJ CGV,20190702,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
4,00303873,CJ CGV,079160,Y,사업보고서 (2017.12),20180330003675,CJ CGV,20180330,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...
5,00303873,CJ CGV,079160,Y,사업보고서 (2016.12),20170331004598,CJ CGV,20170331,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 회사의 재...


In [10]:
# 2016-10-01 ~ 2022-06-01 콘텐트리중앙(메가박스) 회사의 모든 사업보고서

mega=dart.list('036420', start='2016-10-01', end='2022-06-01', kind='A')
mega = mega[mega['report_nm'].str.contains('사업보고서')]
mega

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
1,00203315,콘텐트리중앙,036420,Y,사업보고서 (2021.12),20220323001342,콘텐트리중앙,20220323,
5,00203315,콘텐트리중앙,036420,Y,사업보고서 (2020.12),20210317000814,콘텐트리중앙,20210317,연
9,00203315,콘텐트리중앙,036420,Y,사업보고서 (2019.12),20200330002955,콘텐트리중앙,20200330,연
12,00203315,콘텐트리중앙,036420,Y,[기재정정]사업보고서 (2018.12),20190716000323,콘텐트리중앙,20190717,연
17,00203315,콘텐트리중앙,036420,Y,사업보고서 (2017.12),20180402004265,콘텐트리중앙,20180402,연
21,00203315,콘텐트리중앙,036420,Y,[기재정정]사업보고서 (2016.12),20170515004185,콘텐트리중앙,20170515,연


In [11]:
import time
import random

data_mega=''
data_dict_mega = {}

for key,doc in mega['rcept_no'].iteritems():
    sub_docs = dart.sub_docs(str(doc), match='이사의 경영진단 및 분석의견') # 검색
    url = sub_docs.url[sub_docs.index[0]] # 가장 잘 맞는거 뽑고
    page = requests.get(url) # url뽑고
    time.sleep(random.uniform(0.3,1.8))
    soup = BeautifulSoup(page.content, "html.parser") #페이지 불러오기
    raw_data = soup.find_all('p')
    for item in raw_data:
        data_mega += item.getText()
        data_mega = data_mega.replace(u'\xa0',u'')
        data_dict_mega[doc] = [data_mega] # 딕셔너리 저장


In [12]:
import pandas as pd
megas = pd.DataFrame(data_dict_mega)
megas = megas.transpose()
megas = megas.reset_index()
megas.rename(columns= {0:'text', 'index':'rcept_no'}, inplace=True)
megas

,rcept_no,text
0,20220323001342,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
1,20210317000814,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
2,20200330002955,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
3,20190716000323,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
4,20180402004265,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
5,20170515004185,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...


In [13]:
megas = pd.merge(mega,megas)
megas.to_excel('megas.xlsx')
megas.to_csv("mega.csv", mode='w')
megas

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm,text
0,00203315,콘텐트리중앙,036420,Y,사업보고서 (2021.12),20220323001342,콘텐트리중앙,20220323,,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
1,00203315,콘텐트리중앙,036420,Y,사업보고서 (2020.12),20210317000814,콘텐트리중앙,20210317,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
2,00203315,콘텐트리중앙,036420,Y,사업보고서 (2019.12),20200330002955,콘텐트리중앙,20200330,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
3,00203315,콘텐트리중앙,036420,Y,[기재정정]사업보고서 (2018.12),20190716000323,콘텐트리중앙,20190717,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
4,00203315,콘텐트리중앙,036420,Y,사업보고서 (2017.12),20180402004265,콘텐트리중앙,20180402,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
5,00203315,콘텐트리중앙,036420,Y,[기재정정]사업보고서 (2016.12),20170515004185,콘텐트리중앙,20170515,연,IV. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 주식회사 제이콘텐트...
